In [1]:
%gui qt

In [11]:
from aicspylibczi import CziFile
import imgfile_tools as imf
import czifile_tools as czt
import numpy as np
import zarr
import dask
import dask.array as da
from dask import delayed
from itertools import product
import napari

In [12]:
#filename = r"C:\Temp\input\DTScan_ID4.czi"
filename = r"C:\Users\m1srh\OneDrive - Carl Zeiss AG\Testdata_Zeiss\CZI_Testfiles\S=2_3x3_T=3_Z=4_CH=2.czi"

# get the metadata
md, addmd = imf.get_metadata(filename)
czi = CziFile(filename)
size = czi.read_mosaic_size()

Detected Image Type (based on extension): czi
CZI is Mosaic : True
CZI is RGB : False
No B Dimension : 'SizeB'
No I Dimension : 'SizeI'
No V Dimension : 'SizeV'
DetectorType not found : 'Type'
Trying to extract Scene and Well information if existing ...
Well ArrayNames not found : 'ArrayName'
Well ArrayNames not found : 'ArrayName'
Key not found : 'CustomAttributes'
Scales will not be rounded.


C:\ProgramData\Anaconda3\envs\imageanalysis\lib\site-packages\aicsimageio\transforms.py:178: UserWarning: Data has dimension H with depth 1, assuming H=0 is the desired value, if not the case specify H=x where x is an integer, list, tuple, range, or slice.
  f"Data has dimension {dim} with depth {data.shape[dim_index]}, "
C:\ProgramData\Anaconda3\envs\imageanalysis\lib\site-packages\aicsimageio\transforms.py:178: UserWarning: Data has dimension M with depth 9, assuming M=0 is the desired value, if not the case specify M=x where x is an integer, list, tuple, range, or slice.
  f"Data has dimension {dim} with depth {data.shape[dim_index]}, "


In [13]:
def load_image(czi, md, s=0, t=0, z=0, ch=0):
    
    # get the array for a specifc scene, the BBox and the updated metadata
    scene, bbox, md = czt.read_scene_bbox(czi, md,
                                          sceneindex=s,
                                          channel=ch,
                                          timepoint=t,
                                          zplane=z,
                                          scalefactor=1.0)
    
    return scene

In [16]:
width = md['BBoxes_Scenes'][0].width
height = md['BBoxes_Scenes'][0].height

print(md['SizeS'], md['SizeT'], md['SizeZ'], md['SizeC'], height, width)
sp = [md['SizeT'], md['SizeZ'], md['SizeC'], height, width]

# find the indes for the Scenes dimensions from the dimstring
dims_dict, dimindex_list, numvalid_dims = imf.get_dimorder(md['Axes_aics'])
dims_dict['S']

2 3 4 2 1792 1792


0

In [17]:
# create dask stack of lazy image readers
lazy_process_image = dask.delayed(load_image)  # lazy reader

lazy_arrays = [lazy_process_image(czi, md, s=s, t=t, z=z, ch=ch)
               for s, t, z, ch in product(range(md['SizeS']),
                                          range(md['SizeT']),
                                          range(md['SizeZ']),
                                          range(md['SizeC']))
              ]
dask_arrays = [da.from_delayed(lazy_array,
                               shape=sp,
                               dtype=md['NumPy.dtype'])
               for lazy_array in lazy_arrays
              ]

In [18]:
print(dask_arrays[0].shape)
dask_arrays[1]

(3, 4, 2, 1792, 1792)


dask.array<from-value, shape=(3, 4, 2, 1792, 1792), dtype=uint8, chunksize=(3, 4, 2, 1792, 1792), chunktype=numpy.ndarray>

In [19]:
# concatenate first n array - in this case along the scenes dimension
full_array = da.concatenate(lazy_arrays[:], axis=dims_dict['S'])

ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
import time
viewer = napari.Viewer()
time.sleep(5)

In [ ]:
# configure napari automatically based on metadata
layers = imf.show_napari(viewer, dask_stack, md,
                         blending='additive',
                         gamma=0.85,
                         add_mdtable=True,
                         rename_sliders=True)

In [ ]:
from napari.utils import nbscreenshot
nbscreenshot(viewer)